## 5장 로지스틱 회귀분석

### [1] 이항 분류 예측 실습

- 변수 지정 및 전처리

In [1]:
#1. 모듈 및 함수 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#2. 데이터 불러오기
df = pd.read_csv('Ashopping.csv', encoding = 'cp949')

#3. 변수 지정(독립변수/종속변수)
X = df[['총매출액', '거래기간', '방문빈도']]
Y = df['이탈여부']

#4. 데이터 분할(학습용/평가용 데이터 세트)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#5. 표준화 
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print(X_train)

[[ 0.69336175  0.22690456  0.03475125]
 [-0.225208   -0.02778804 -0.17540748]
 [-0.52055725  0.61729402  0.98046554]
 ...
 [-0.60960507  0.55779616 -0.01778844]
 [-0.02842033 -0.64468689 -0.22794717]
 [-0.30394592  0.83127755 -0.70080431]]


In [2]:
#1. 모듈 및 함수 불러오기
from imblearn.over_sampling import SMOTE
from collections import Counter

#2 오버 샘플링 
smote = SMOTE(random_state=0)
X_train_over, Y_train_over = smote.fit_sample(X_train, Y_train)

#3. 결과 출력
print('Original dataset shape %s' % Counter(Y))
print('sampled dataset shape %s' % Counter(Y_train))
print('Resampled dataset shape %s' % Counter(Y_train_over))


Using TensorFlow backend.


Original dataset shape Counter({0: 700, 1: 300})
sampled dataset shape Counter({0: 481, 1: 219})
Resampled dataset shape Counter({1: 481, 0: 481})


- 모형 학습 및 예측

In [3]:
#1. 모듈 및 함수 불러오기
from sklearn.linear_model import LogisticRegression

#2 모형 생성 
model = LogisticRegression(C = 1 ,random_state=0) 

#3 모형 학습 및 예측
model.fit(X_train_over, Y_train_over)
Y_pred = model.predict(X_test)
print('평가용 데이터 세트에 대한 예측 값\n', Y_pred)


평가용 데이터 세트에 대한 예측 값
 [1 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 1 1 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 1 1 0 1 1 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1
 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1 1
 1 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 0
 1 1 1 0 1 1 0 1 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0
 0 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 0 1 0 1 1 1 0 0 1 0
 0 0 0 0 0 1 0 1 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 1 0 1 0 0 0 1
 0 1 0 1]


- 모형 평가

In [4]:
print('학습용 데이터 세트 정확도: {:.3f}'.format(model.score(X_train_over, Y_train_over)))
print('평가용 데이터 세트 정확도: {:.3f}'.format(model.score(X_test, Y_test)))


학습용 데이터 세트 정확도: 0.710
평가용 데이터 세트 정확도: 0.703


In [5]:
#1. 모듈 및 함수 불러오기
from sklearn.metrics import classification_report

#2. 정밀도, 재현율, F1 스코어 출력
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.91      0.66      0.77       219
           1       0.47      0.81      0.60        81

    accuracy                           0.70       300
   macro avg       0.69      0.74      0.68       300
weighted avg       0.79      0.70      0.72       300



- 회귀계수 출력

In [6]:
print('총매출액 회귀계수: {0:.3f}, 거래기간 회귀계수: {1:.3f},방문빈도 회귀계수: {2:.3f}'.format(model.coef_[0,0], model.coef_[0,1], model.coef_[0,2]))

총매출액 회귀계수: -0.107, 거래기간 회귀계수: -0.584,방문빈도 회귀계수: -0.479


### [2] 다항 로지스틱 회귀분석

- 변수 지정 및 전처리

In [7]:
#1. 모듈 및 함수 불러오기
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#2. 변수 지정(독립변수/종속변수) 
X =df[['방문빈도', '총 할인 금액', '거래기간', '할인민감여부']]
Y = df['구매금액대']

#.3 데이터 분할(학습용/ 평가용 데이터 세트)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)


#4. 표준화 및 원핫인코딩
ct = ColumnTransformer([('scaling', StandardScaler(), ['방문빈도', '총 할인 금액', '거래기간']),
                        ('onehot', OneHotEncoder(sparse = False),['할인민감여부'])])
ct.fit(X_train)
X_train=ct.transform(X_train)
X_test=ct.transform(X_test)
X_test

array([[-0.4381059 ,  0.52039092, -0.72610502,  0.        ,  1.        ],
       [ 0.03475125,  0.6181886 , -0.57683846,  0.        ,  1.        ],
       [-0.96350273,  0.8044049 ,  0.42836222,  1.        ,  0.        ],
       ...,
       [-0.28048685,  0.67190908, -1.04760222,  1.        ,  0.        ],
       [ 0.24490998, -2.47210367, -0.37224933,  1.        ,  0.        ],
       [-0.28048685,  0.80004562, -2.15614129,  1.        ,  0.        ]])

- 모형 학습 및 예측

In [12]:
#1. 모형 생성
model=LogisticRegression(random_state=0,C=0.1,solver='newton-cg', multi_class='multinomial')

#2. 모형 학습 및 예측
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print('평가용 데이터 세트에 대한 예측값\n', Y_pred)

평가용 데이터 세트에 대한 예측값
 [0 2 0 2 2 2 1 0 0 0 0 0 1 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 1 0 2 0 0 0
 0 2 0 0 0 0 0 0 1 0 0 0 1 2 0 0 0 2 0 0 0 2 2 0 2 0 0 0 0 2 0 0 1 2 0 0 0
 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 1 0 0 0 0 0 2 2 1 0 0 1 0 0 2 0 2 2 2 2 0 0
 2 0 0 0 0 0 2 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 0 0
 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1 0 0 1 0 0 0 0 0 2 0 2 0 0 0 0
 0 0 2 0 0 0 0 0 0 0 1 2 0 0 0 0 1 0 1 2 1 2 0 0 0 1 0 0 0 0 0 0 0 0 0 2 0
 1 0 0 2 0 0 0 2 0 0 0 0 0 0 2 0 0 2 0 2 0 0 2 1 2 0 2 0 0 0 0 0 0 0 0 0 0
 0 2 1 1 1 0 0 0 0 2 0 2 0 0 0 0 1 0 0 0 0 0 2 0 0 0 0 2 0 0 0 0 0 0 0 0 0
 0 0 0 0]


- 모형 평가

In [13]:
#1. 정확도 평가
print('학습용 데이터 세트 정확도: {:.3f}'.format(model.score(X_train, Y_train)))
print('평가용 데이터 세트 정확도: {:.3f}'.format(model.score(X_test, Y_test)))


학습용 데이터 세트 정확도: 0.606
평가용 데이터 세트 정확도: 0.607


In [14]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.61      0.89      0.72       154
           1       0.40      0.12      0.19        82
           2       0.71      0.55      0.62        64

    accuracy                           0.61       300
   macro avg       0.57      0.52      0.51       300
weighted avg       0.57      0.61      0.55       300



- 회귀계수 출력

In [11]:
print('---구매금액대 0의 회귀계수---')
print('방문빈도 회귀계수: {0:.3f}, 총 할인 금액 회귀계수: {1:.3f}, 거래기간: {2:.3f}, 할인민감여부_1: {3:.3f}'.format(model.coef_[0,0], model.coef_[0,1], model.coef_[0,2], model.coef_[0,4]))

print('---구매금액대 1의 회귀계수---')
print('방문빈도 회귀계수: {0:.3f}, 총 할인 금액 회귀계수: {1:.3f}, 거래기간: {2:.3f}, 할인민감여부_1: {3:.3f}'.format(model.coef_[1,0], model.coef_[1,1], model.coef_[1,2], model.coef_[1,4]))

print('---구매금액대 2의 회귀계수---')
print('방문빈도 회귀계수: {0:.3f}, 총 할인 금액 회귀계수: {1:.3f}, 거래기간: {2:.3f}, 할인민감여부_1: {3:.3f}'.format(model.coef_[2,0], model.coef_[2,1], model.coef_[2,2], model.coef_[2,4]))


---구매금액대 0의 회귀계수---
 방문빈도 회귀계수: -0.632, 총 할인 금액 회귀계수: -0.037, 거래기간: 0.113, 할인민감여부_1: -0.435
---구매금액대 1의 회귀계수---
 방문빈도 회귀계수: 0.089, 총 할인 금액 회귀계수: 0.013, 거래기간: 0.088, 할인민감여부_1: -0.223
---구매금액대 2의 회귀계수---
 방문빈도 회귀계수: 0.543, 총 할인 금액 회귀계수: 0.024, 거래기간: -0.201, 할인민감여부_1: 0.658
